Adapted from https://github.com/milesial/Pytorch-UNet and the PyTorch documentation.

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import matplotlib
import matplotlib.pyplot as plt
import gc, collections, resource

In [2]:
from unet_model import UNet
from dataset import PandoraImage
from eval import eval_net

In [3]:
#%load_ext tensorboard
!rm -rf ./runs/
#%tensorboard --logdir /runs

rm: cannot remove './runs/May24_15-45-32_x360LR_0.0001_BS_1': Directory not empty
rm: cannot remove './runs/May24_15-51-50_x360LR_0.0001_BS_1': Directory not empty
rm: cannot remove './runs/May24_15-52-24_x360LR_0.0001_BS_1': Directory not empty
rm: cannot remove './runs/May24_15-59-03_x360LR_0.0001_BS_1': Directory not empty
rm: cannot remove './runs/May24_16-14-04_x360LR_0.0001_BS_1': Directory not empty
rm: cannot remove './runs/May24_16-23-59_x360LR_0.0001_BS_1': Directory not empty
rm: cannot remove './runs/May24_16-37-11_x360LR_0.0001_BS_1': Directory not empty
rm: cannot remove './runs/May24_16-51-14_x360LR_0.0001_BS_1': Directory not empty
rm: cannot remove './runs/May24_22-30-50_x360LR_0.0001_BS_1': Directory not empty
rm: cannot remove './runs/May24_22-33-18_x360LR_0.0001_BS_1': Directory not empty


In [4]:
# https://forum.pyro.ai/t/a-clever-trick-to-debug-tensor-memory/556
def debug_memory():
    print('maxrss = {}'.format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss))
    tensors = collections.Counter((str(o.device), o.dtype, tuple(o.shape))
                                  for o in gc.get_objects()
                                  if torch.is_tensor(o))
    for line in tensors.items():
        print('{}\t{}'.format(*line))

In [5]:
def NoBackgroundCrossEntropyLoss(masks_pred, true_masks):
    print("torch.shape(masks_pred)", masks_pred.size())
    print("torch.shape(true_masks)", true_masks.size())
    x = torch.sum(true_masks, dim=1)
    print("torch.shape(x)", x.size())
    x = x[:,None,:,:]
    x = x.repeat_interleave(3,1)
    background_mask = (x==0.0)
    print("torch.shape(background_mask)", background_mask.size())
    masks_pred[background_mask] = 0.0
    print("torch.shape(masks_pred)2", masks_pred.size())
    #return nn.CrossEntropyLoss()(masks_pred, true_masks).cuda()
    return nn.BCEWithLogitsLoss()(masks_pred, true_masks).cuda()

In [6]:
def train_net(net, device, dataPath, epochs=5, batch_size=1, lr=0.001, val_percent=0.1, save_cp=True):
    global_step = 0
    
    print("DataInput")
    dataset = PandoraImage(dataPath, imsize=384)
    n_val = int(len(dataset) * val_percent)
    n_train = len(dataset) - n_val
    train, val = random_split(dataset, [n_train, n_val])
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True, drop_last=True)
    
    writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}')
    
    optimizer = optim.RMSprop(net.parameters(), lr=lr, weight_decay=1e-8, momentum=0.9)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)
    criterion = nn.CrossEntropyLoss(ignore_index=3)
    
    for epoch in range(epochs):
        net.train()
        epoch_loss = 0
        with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img') as pbar:
            for batch in train_loader:
                imgs = batch['image']
                true_masks = batch['mask']
                imgs = imgs.to(device=device, dtype=torch.float32)
                true_masks = true_masks.to(device=device, dtype=torch.long) # Why long???

                masks_pred = net(imgs)
                loss = criterion(masks_pred, true_masks)#NoBackgroundCrossEntropyLoss(masks_pred, true_masks)
                epoch_loss += loss.item()
                
                writer.add_scalar('Batch Loss (train)', loss.item(), global_step)
                
                pbar.set_postfix(**{'loss (batch)': loss.item()})
                pbar.update(imgs.shape[0])
                optimizer.zero_grad()

                loss.backward()
                nn.utils.clip_grad_value_(net.parameters(), 0.1) # Necessary ???
                optimizer.step()

                global_step += 1
                if global_step % (len(dataset) // (10 * batch_size)) == 0:
                    for tag, value in net.named_parameters():
                        tag = tag.replace('.', '/')
                        writer.add_histogram('weights/' + tag, value.data.cpu().numpy(), global_step)
                        writer.add_histogram('grads/' + tag, value.grad.data.cpu().numpy(), global_step)
                    val_score = eval_net(net, val_loader, device)
                    scheduler.step(val_score)
                    writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], global_step)
                    writer.add_scalar('Loss/test', val_score, global_step)
                    writer.add_images('images', imgs, global_step)
                    try:
                        writer.add_images('masks/true', true_masks, global_step, dataformats='NCHW')
                        writer.add_images('masks/pred', masks_pred, global_step, dataformats='NCHW')
                    except:
                        print("Problem adding masks")
                        
            
        if save_cp:
            torch.save(net.state_dict(), 'checkpoints/' + f'CP_epoch{epoch + 1}.pth')
    writer.close()

# Training

In [7]:
#debug_memory()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = UNet(n_channels=1, n_classes=3, bilinear=False)
net.to(device=device)

path = "/home/philip/RemoteFNAL/PanLee_v8_00_00_13/OutTest/viewV_May13.bin"

train_net(net=net,
          dataPath = path,
          epochs=5,
          batch_size=1,
          lr=0.0001,
          device=device,
          val_percent=0.2)

DataInput


Validation round:  16%|█▌        | 110/703 [00:24<02:05,  4.74batch/s]


Validation round:  30%|███       | 213/703 [00:53<02:08,  3.82batch/s]

nextSep = self.data.size-1;
21573759
None



Validation round:  46%|████▌     | 323/703 [01:20<01:27,  4.34batch/s]


...

Validation round:  77%|███████▋  | 543/703 [02:15<00:38,  4.10batch/s]


Validation round:  93%|█████████▎| 653/703 [02:42<00:11,  4.55batch/s]


Validation round: 100%|██████████| 703/703 [02:53<00:00,  4.04batch/s]


Problem adding masks


Epoch 1/5:  69%|██████▉   | 1945/2815 [42:57<13:23,  1.08img/s, loss (batch)=0.87]    

In [ ]:
#del net, device
#gc.collect()

In [ ]:
dataset = PandoraImage(path, imsize=384)
n_val = int(len(dataset) * 0.2)
n_train = len(dataset) - n_val
train, val = random_split(dataset, [n_train, n_val])
train_loader = DataLoader(train, batch_size=1, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val, batch_size=1, shuffle=False, num_workers=2, pin_memory=True, drop_last=True)

In [ ]:
net.eval()

a = iter(val_loader)
next(a)
next(a)
batch = next(a)

#for batch in val_loader:
imgs, true_masks = batch['image'], batch['mask']

with torch.no_grad():
    mask_pred = torch.sigmoid(net(imgs.float()))


plt.figure(figsize=(20, 30))

cmap = matplotlib.cm.viridis  # Can be any colormap that you want after the cm
cmap.set_bad(color='black')

plt.subplot(3, 2, 1)
plt.title("Image")
plt.imshow(np.ma.masked_where(imgs[0,0,:,:] == 0, imgs[0,0,:,:]), cmap=cmap)

plt.subplot(3,2, 2)
plt.title("true Mask")
plt.imshow(np.ma.masked_where(true_masks[0,:,:] == 3, true_masks[0,:,:]), cmap=cmap)
plt.clim(0, 2)
plt.colorbar()

plt.subplot(3,2, 3)
plt.title("Raw Predicted Mask")
mask2 = mask_pred.permute(0, 2, 3, 1)
plt.imshow(mask2[0,:,:,:])


#maskSum = np.sum(mask_pred[0,:,:,:], axis=0)
flat_pred_mask = np.argmax(mask_pred[0,:,:,:], axis=0)
#maskBkgd = np.where(maskSum==0.0)
#mask3[maskBkgd]=3
plt.subplot(3,2,4)
plt.title("Predicted Mask")
plt.imshow(flat_pred_mask)
plt.clim(-2, 3)
plt.colorbar()

print(true_masks.size())
print(flat_pred_mask.size())
interestMask = np.where(true_masks[0,:,:]==3)
clean_flat_pred_mask = flat_pred_mask
clean_flat_pred_mask[interestMask]=0
print(clean_flat_pred_mask.size())
diffMask=true_masks[0,:,:]-clean_flat_pred_mask
print(diffMask.size())


plt.subplot(3,2,5)
plt.title("Clean Prediction Mask")
plt.imshow(np.ma.masked_where(true_masks[0,:,:] == 3, flat_pred_mask), cmap=cmap)
plt.clim(0, 2)
plt.colorbar()

plt.subplot(3,2,6)
plt.title("Difference Mask")
plt.imshow(np.ma.masked_where(np.logical_or(diffMask == 3, diffMask == 0), diffMask), cmap=cmap)
plt.clim(-2, 2)
plt.colorbar()

plt.show()